This notebook looks at the influence function of the logarithm of the score matching density function
in a kernel exponential family $\mathcal{Q}$ evaluated at a point $w$, which is defined as 

$$
\mathrm{IF} \big( y; \log q (w; F) \big) := \lim_{\varepsilon \to 0^+} \frac{1}{\varepsilon} \Big(\log q \big(w; (1 - \varepsilon) F + \varepsilon \delta_y \big) - \log q \big(w; F\big)\Big), \quad \text{ for all } w \in \mathcal{X}, \hspace{50pt} (*)
$$

where $\mathcal{X} \subseteq \mathbb{R}$ is the sample space, $F$ is a probability distribution over $\mathcal{X}$, $q (\cdot; F): \mathcal{X} \to [0, \infty)$ is the score matching density function in $\mathcal{Q}$ under $F$, $\varepsilon \in (0, 1]$, and $\delta_y$ is the point mass 1 at $y \in \mathcal{X}$. 

We approximate $(*)$ by 

$$
\widehat{\mathrm{IF}} \big( y; \log q (w; F_n) \big) := \frac{1}{\varepsilon} \Big(\log q \big(w; (1 - \varepsilon) F_n + \varepsilon \delta_y\big) - \log q \big(w; F_n\big)\Big), \quad \text{ for all } w \in \mathcal{X}, \hspace{50pt} (**)
$$

with a small $\varepsilon$, where $F_n$ is the empirial distribution. 

In the below, we use the `waiting` variable in the Old Faithful Geyser dataset and insert an additional observation, i.e., $y$ in $(**)$, each time. These additional observations are $90$, $92$, $\cdots$, $398$, $400$. In additional, we choose the sample space $\mathcal{X} = (0, \infty)$, the kernel function to be the Gaussian kernel function, the bandwidth parameter to be $5.0$, $7.0$ and $9.0$, the penalty parameter to be $\exp({-12.0})$, $\exp({-10.0})$ and $\exp({-8.0})$, and $\varepsilon$ in $(*)$ to be `1e-8`. 

In [2]:
import os 
import copy 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
import seaborn as sns

from matplotlib.animation import FuncAnimation
%matplotlib qt
# %matplotlib inline 

In [3]:
os.chdir('/Users/chenxizhou/Dropbox/code_package/IFlogdensity')

# kernel_type = 'gaussian_poly2'
# bw_list = [5.0] # [5.0, 7.0, 9.0]
# log_pen_param_list = [-12.0] # [-12.0, -10.0, -8.0]
# contam_weight = 1e-8

We look the resulting influence function $(**)$ evaluated at $w \in \mathcal{X}$ as we move the additional observation. 

In [ ]:
for bw in bw_list: 
    for log_pen_param in log_pen_param_list: 
        print('=' * 100)
        print(f'bw={bw}, pen={log_pen_param}')
        save_dir=f'data/bw={bw}-kernel={kernel_type}-loglambda={log_pen_param}-contamweight={contam_weight}'

        contam_data_array = np.array([106.]).reshape(-1, 1) # np.arange(90., 402., 20).reshape(-1, 1)
        new_data = np.load(save_dir + f'/new_data.npy').flatten()

        linewidth = 2.0
        label_fontsize = 20
        info_fontsize = 25
        tick_fontsize = 15
        linewidth = 3.0
        x_label = 'waiting'
        y_label = 'IF'

        for i in range(len(contam_data_array)): 
            # read in data 
            save_dir = f'data/bw={bw}-kernel={kernel_type}-loglambda={log_pen_param}-contamweight={contam_weight}'

            result = np.load(save_dir + f'/contam_data={contam_data_array[i]}-IF-logden-newdata.npy')

            fig = plt.figure(figsize=(20, 10))
            left, bottom, width, height = 0.1, 0.1, 0.8, 0.8
            ax = fig.add_axes([left, bottom, width, height])

            plt.plot(new_data.flatten(), result, color = 'tab:blue', linewidth = linewidth)
            ax.axhline(0., 0, 1, ls = '--', color = 'tab:purple', alpha = 0.5)

            plt.title('Influence Function of Score Matching Log-density with $\sigma$={bw} and $\lambda$=exp({pen})'.format(
    bw=bw, pen=log_pen_param), fontsize = info_fontsize)
            plt.xlabel('waiting', fontsize = label_fontsize)
            plt.ylabel('IF', fontsize = label_fontsize)
            plt.xlim((21., 410.))
            
            ax.tick_params(axis = 'both', labelsize = tick_fontsize)
            
            info = r"Add {add_obs}".format(
                bw = bw, pen = log_pen_param, add_obs = contam_data_array[i][0])
            ax.text(0.995, 0.985,
                    info,
                    fontsize = info_fontsize,
                    multialignment = 'left',
                    horizontalalignment = 'right',
                    verticalalignment = 'top',
                    transform = ax.transAxes,
                    bbox = {'facecolor': 'none',
                            'boxstyle': 'Round, pad=0.2'})
            plt.savefig('plots/IF-plot-L2norm=inf.pdf')
            plt.show()
            

In [15]:
os.chdir('/Users/chenxizhou/Dropbox/code_package/IFlogdensity')

true_data = np.load('data/geyser.npy').astype(np.float64)
df = copy.deepcopy(true_data[:, 0]).reshape(-1, 1)
data_waiting = df[df != 108.0]

xlimit = (1., 310.)
plot_pts_cnt = 3000
newx = np.linspace(xlimit[0], xlimit[1], plot_pts_cnt)

contam_data_array = np.arange(5., 310., 5)# .reshape(-1, 1)
contam_weight = 1e-3

kernel_type = 'gaussian_poly2'
bw = 9.0
log_pen_param = -10.0
ylimit = (-1600., 1600.)

var_name = 'additional observation'
method = 'SM'

fontsize_label = 15
fontsize_tick = 10
fontsize_info = 20
fontsize_title = 20
fontsize_suptitle = 22
linewidth = 2.0

fig, ax = plt.subplots(
    nrows = 1, 
    ncols = 1, 
    figsize = (20, 10), 
    # tight_layout = True, 
    constrained_layout = False)

fig.subplots_adjust(top=0.9)

def update_IF_plots(contam_data): 

    # read in the original data 
    true_data = np.load('data/geyser.npy').astype(np.float64)
    df = copy.deepcopy(true_data[:, 0]).reshape(-1, 1)
    df = df[df != 108.0]
    
    pddf = pd.DataFrame({'data': df.flatten()})
    
    ax.clear()
    
    # ---------------------------------------------------------------------------------------
    # set x-limit 
    ax.set_xlim(xlimit)
    # set x label 
    ax.set_xlabel(var_name, fontsize = fontsize_label)
    # set y label 
    ax.set_ylabel('IF', fontsize = fontsize_label)
    ax.set_ylim(ylimit)
    # formatting tick marks and tick labels 
    ax.tick_params(axis = 'both', labelsize = fontsize_tick)
    ax.ticklabel_format(axis = 'y')
    # add rug plot at normal observations 
    sns.rugplot(pd.Series(df.flatten()), ax = ax, color = 'tab:blue')
    sns.rugplot(pd.Series(contam_data), ax = ax, color = 'red')
    
    save_dir = (f'data/PenSM-FinKEF-bw={bw}-kernel={kernel_type}-logpenparam={log_pen_param}-' + 
                f'contamweight={contam_weight}-plotdomain=(1.0, 310.0)-plotcnts=3000')
    result = np.load(save_dir + f'/contam_data={contam_data}-IF-newdata.npy')
#     result_lim = np.load(
#         save_dir + f'/contam_data={contam_data}-IF-natparam-limit-newdata.npy')
    
    # plot density when the basis functions are centered at grid points 
    ax.plot(newx.flatten(), result.flatten(), color = 'tab:blue', 
            linestyle = 'solid', linewidth = linewidth, label = 'IF of Log-density')
#     ax.plot(newx.flatten(), result_lim.flatten(), color = 'tab:red', 
#             linestyle = 'dashed', linewidth = linewidth, label = 'Limiting Case', alpha = 0.8)
    plt.legend(fontsize = fontsize_info, loc = 'upper right')
    
    # draw a vertical line at the outlier 
    ax.axvline(contam_data, 0, 1, ls = '--', color = 'tab:purple', alpha = 0.5)

    # add plot information 
    info = f'Add {contam_data}'
    ax.text(0.007, 0.988,
            info,
             fontsize = fontsize_info,
             # fontfamily = 'serif',
             multialignment = 'left',
             horizontalalignment = 'left',
             verticalalignment = 'top',
             transform = ax.transAxes,
             bbox = {'facecolor': 'none',
                     'boxstyle': 'Round, pad=0.2'})
    
    return ax

ani = FuncAnimation(
    fig, 
    update_IF_plots, 
    frames = contam_data_array, 
    interval = 200)

fig.suptitle(r'Influence Function of Score Matching Log-density with $\sigma$={bw} and $\lambda$=exp({pen})'.format(
    bw=bw, pen=log_pen_param), 
             fontsize = fontsize_suptitle, y = 0.98)

# uncomment the following line to save the gif
# ani.save(f'gif/{method}-IF-logdensity-limit-waiting-kernel={kernel_type}-bw={bw}-pen=exp{log_pen_param}-contamweight={contam_weight}.gif', writer='imagemagick')

plt.show()

In [ ]:
PenSM-FinKEF-bw=9.0-kernel=gaussian_poly2-logpenparam=-10.0-contamweight=0.001-plotdomain=(1.0, 310.0)-plotcnts=3000
PenSM-FinKEF-bw=9.0-kernel=gaussian_poly2-logpenparam=-10.0-contamweight=0.001-plotdomain=(1.0, 310.0)-plotcnts=3000

In [ ]:
os.chdir('/Users/chenxizhou/Dropbox/code_package/IFlogdensity')

true_data = np.load('data/geyser.npy').astype(np.float64)
df = copy.deepcopy(true_data[:, 0]).reshape(-1, 1)
data_waiting = df[df != 108.0]

xlimit = (1., 410.) # (21., 410.)
plot_pts_cnt = 3000 # 2000
newx = np.linspace(xlimit[0], xlimit[1], plot_pts_cnt)

contam_data_array = np.arange(2., 410., 4).reshape(-1, 1)
contam_weight = 0.01

kernel_type = 'gaussian_poly2'
bw = 9.0
log_pen_param = -8.0
stepsize = 0.3
ylimit = (-79., 179.)
seed = 0

var_name = 'additional observation'
method = 'PenML'

fontsize_label = 15
fontsize_tick = 10
fontsize_info = 20
fontsize_title = 20
fontsize_suptitle = 22
linewidth = 3.0

fig, ax = plt.subplots(
    nrows = 1, 
    ncols = 1, 
    figsize = (20, 10), 
    # tight_layout = True, 
    constrained_layout = False)

fig.subplots_adjust(top=0.9)

def update_IF_plots(contam_data): 

    # read in the original data 
    true_data = np.load('data/geyser.npy').astype(np.float64)
    df = copy.deepcopy(true_data[:, 0]).reshape(-1, 1)
    df = df[df != 108.0]
    
    pddf = pd.DataFrame({'data': df.flatten()})
    
    ax.clear()
    
    # ---------------------------------------------------------------------------------------
    # set x-limit 
    ax.set_xlim(xlimit)
    # set x label 
    ax.set_xlabel(var_name, fontsize = fontsize_label)
    # set y label 
    ax.set_ylabel('IF', fontsize = fontsize_label)
    ax.set_ylim(ylimit)
    # formatting tick marks and tick labels 
    ax.tick_params(axis = 'both', labelsize = fontsize_tick)
    ax.ticklabel_format(axis = 'y')
    # add rug plot at normal observations 
    sns.rugplot(pd.Series(df.flatten()), ax = ax, color = 'tab:blue')
    sns.rugplot(pd.Series(contam_data), ax = ax, color = 'red')
    
    save_dir = (f'data/PenML-basisn=205-bw={bw}-' + 
                f'kernel={kernel_type}-loglambda={log_pen_param}-contamweight={contam_weight}-' + 
                f'plotdomain={xlimit}-plotcnts={plot_pts_cnt}-abstol={0.05}-stepsize={stepsize}-seed={seed}')
        
    result = np.load(save_dir + f'/contam_data={contam_data}-IF-logden-newdata.npy')

    # plot density when the basis functions are centered at grid points 
    ax.plot(newx.flatten(), result.flatten(), color = 'tab:blue', 
            linestyle = 'solid', linewidth = linewidth)

    # draw a vertical line at the outlier 
    ax.axvline(contam_data, 0, 1, ls = '--', color = 'tab:purple', alpha = 0.5)

    # add plot information 
    info = f'Add {contam_data[0]}'
    ax.text(0.007, 0.988,
            info,
             fontsize = fontsize_info,
             # fontfamily = 'serif',
             multialignment = 'left',
             horizontalalignment = 'left',
             verticalalignment = 'top',
             transform = ax.transAxes,
             bbox = {'facecolor': 'none',
                     'boxstyle': 'Round, pad=0.2'})
    
    return ax

ani = FuncAnimation(
    fig, 
    update_IF_plots, 
    frames = contam_data_array, 
    interval = 200)

fig.suptitle(r'Influence Function of Maximum Likelihood Log-density with $\sigma$={bw} and $\lambda$=exp({pen})'.format(
    bw=bw, pen=log_pen_param), 
             fontsize = fontsize_suptitle, y = 0.98)

# uncomment the following line to save the gif
ani.save(f'gif/{method}-IF-logdensity-waiting-kernel={kernel_type}-bw={bw}-pen=exp{log_pen_param}-contamweight={contam_weight}.gif', writer='imagemagick')

plt.show()

### Fix $\sigma$ and $\lambda$, vary contaminated observation

In [ ]:
os.chdir('/Users/chenxizhou/Dropbox/code_package/IFlogdensity')

bw = 9.0
log_pen_param = -12.
contam_data_list = np.arange(5., 305., 5).reshape(-1, 1)
xlimit = (1., 310.)
plot_pts_cnt = 3000
seed = 1
var_name = 'waiting'
title_fontsize = 25
label_fontsize = 20
info_fontsize = 25
tick_fontsize = 18
fontsize_suptitle = 25
linewidth = 3.0
contam_weight = 0.001
ylimit1 = (-129., 249)
ylimit2 = (-3599., 4399.) # (-499., 659.) #  # ylimit1

fig, (ax1, ax2) = plt.subplots(
    nrows = 1, 
    ncols = 2, 
    figsize = (20, 40), 
    # tight_layout = True, 
    constrained_layout = False)

fig.subplots_adjust(top=0.9)

# func
def update_plots(outlier): 

    # read in the original data 
    true_data = np.load('data/geyser.npy').astype(np.float64)
    df = copy.deepcopy(true_data[:, 0]).reshape(-1, 1)
    df[df == 108.0] = outlier
    
    pddf = pd.DataFrame({'vals': df.flatten(),
                         'cate': [False if df[i] != outlier else True for i in range(df.shape[0])]})
    
    ax1.clear()
    ax2.clear()
    
    # ---------------------------------------------------------------------------------------
    # set ax1 title 
    # ax1.set_title('Maximum Likelihood', fontsize = title_fontsize)
    # set x-limit 
    ax1.set_xlim(xlimit)
    # set x label 
    ax1.set_xlabel(var_name, fontsize = label_fontsize)
    # set y label 
    ax1.set_ylabel('IF', fontsize = label_fontsize)
    # formatting tick marks and tick labels 
    ax1.tick_params(axis = 'both', labelsize = tick_fontsize)
#     ax1.ticklabel_format(axis = 'y', style = 'sci', scilimits = scilimits)
    # add rug plot at normal observations 
    sns.rugplot(pddf['vals'], ax = ax1, color = 'tab:blue')
    sns.rugplot(np.array([outlier.item()]), ax = ax1, color = 'red')

    save_dir_ml = (f'data/PenML-GD-basisn=310-bw={bw}-kernel={kernel_type}-log_pen_param={log_pen_param}-' + 
                   f'contamweight={contam_weight}-plotdomain={xlimit}-plotcnts={plot_pts_cnt}-seed={seed}')
    
    x = np.load(save_dir_ml + '/new_data.npy')
    y = np.load(save_dir_ml + f'/contam_data={outlier.item()}-IF-newdata.npy')
    
    # plot log density when the basis functions are centered at grid points 
    ax1.plot(x.flatten(), y.flatten(), color = 'tab:red', linewidth = linewidth, 
            label = 'Maximum Likelihood')
    
    # draw a vertical line at the outlier 
    ax1.axvline(outlier, 0, 1, ls = '--', color = 'tab:purple', alpha = 0.5)
    ax1.set_ylim(ylimit1)
    
#     # add grid
#     ax1.grid(color = 'k', ls = (0, (3, 10, 1, 10)), lw = 0.25)

    # add plot information 
    info = f'Add {outlier.item()}'
    ax1.text(0.988, 0.988,
             info,
             fontsize = info_fontsize,
             # fontfamily = 'serif',
             multialignment = 'left',
             horizontalalignment = 'right',
             verticalalignment = 'top',
             transform = ax1.transAxes,
             bbox = {'facecolor': 'none',
                     'boxstyle': 'Round, pad=0.2'})

    # ---------------------------------------------------------------------------------------
    # set ax2 title 
    # ax2.set_title('Basis Functions Centered at Data Points', fontsize = fontsize_title)
    # set x-limit 
    ax2.set_xlim(xlimit)
    # set x label 
    ax2.set_xlabel(var_name, fontsize = label_fontsize)
    # set y label 
    ax2.set_ylabel('IF', fontsize = label_fontsize)
    # formatting tick marks and tick labels 
    ax2.tick_params(axis = 'both', labelsize = tick_fontsize)
    # add rug plot at normal observations 
    sns.rugplot(pddf['vals'], ax = ax2, color = 'tab:blue')
    sns.rugplot(np.array([outlier.item()]), ax = ax2, color = 'red')
    
    save_dir_sm = (f'data/PenSM-bw={bw}-kernel={kernel_type}-loglambda={log_pen_param}-contamweight={contam_weight}-' + 
                   f'plotdomain={xlimit}-plotcnts={plot_pts_cnt}')
    
    x = np.load(save_dir_sm + '/new_data.npy')
    y = np.load(save_dir_sm + f'/contam_data={outlier}-IF-logden-newdata.npy')
    
    
    ax2.plot(x.flatten(), y.flatten(), color = 'tab:blue', linewidth = linewidth, 
            label = 'Score Matching')
    
    # draw a vertical line at the outlier 
    ax2.axvline(outlier, 0, 1, ls = '--', color = 'tab:purple', alpha = 0.5)
    ax2.set_ylim(ylimit2)
    
#     # add grid
#     ax2.grid(color = 'k', ls = (0, (3, 10, 1, 10)), lw = 0.25)
    
    # add plot information 
    info = f'Add {outlier.item()}'
    ax2.text(0.988, 0.988,
             info,
             # fontfamily = 'serif',
             fontsize = info_fontsize,
             multialignment = 'left',
             horizontalalignment = 'right',
             verticalalignment = 'top',
             transform = ax2.transAxes,
             bbox = {'facecolor': 'none',
                     'boxstyle': 'Round, pad=0.2'})
    
    return ax1, ax2

ani = FuncAnimation(
    fig, 
    update_plots, 
    frames = contam_data_list, 
    interval = 500)

fig.suptitle(r'IF of ML and SM Log-density Estimates with $\sigma$={bw} and $\lambda$=exp({logpen})'.format(
    bw=bw, logpen=log_pen_param), 
             fontsize = fontsize_suptitle, y = 0.98)

# uncomment the following line to save the gif
# ani.save(f'gif/PenML-PenSM-waiting-bw={bw}-pen=exp({log_pen_param}).gif', writer='imagemagick')

plt.show()

### Fix $\sigma$ and contaminated observation, vary $\lambda$

In [16]:
os.chdir('/Users/chenxizhou/Dropbox/code_package/IFlogdensity')

bw = 9.0
log_pen_param_list = np.concatenate((np.array([-15., -14., -13., -12., -11.]), np.arange(-10., 1.1, 0.5)))
contam_data = 120.

xlimit = (1., 310.)
plot_pts_cnt = 3000
seed = 1
var_name = 'waiting'
kernel_type = 'gaussian_poly2'
title_fontsize = 25
label_fontsize = 20
info_fontsize = 25
tick_fontsize = 18
fontsize_suptitle = 25
linewidth = 3.0
contam_weight = 0.001
ylimit1 = (-219., 219)
ylimit2 = (-2299., 2299.) # (-499., 659.) #  # ylimit1

fig, (ax1, ax2) = plt.subplots(
    nrows = 1, 
    ncols = 2, 
    figsize = (20, 40), 
    # tight_layout = True, 
    constrained_layout = False)

fig.subplots_adjust(top=0.9)

# func
def update_plots(log_pen_param): 

    # read in the original data 
    true_data = np.load('data/geyser.npy').astype(np.float64)
    df = copy.deepcopy(true_data[:, 0]).reshape(-1, 1)
    df = df[df != 108.]
    
    pddf = pd.DataFrame({'vals': df.flatten(),
                         'cate': [False if df[i] != contam_data else True for i in range(df.shape[0])]})
    
    ax1.clear()
    ax2.clear()
    
    # ---------------------------------------------------------------------------------------
    # set ax1 title 
    # ax1.set_title('Maximum Likelihood', fontsize = title_fontsize)
    # set x-limit 
    ax1.set_xlim(xlimit)
    # set x label 
    ax1.set_xlabel(var_name, fontsize = label_fontsize)
    # set y label 
    ax1.set_ylabel('IF', fontsize = label_fontsize)
    # formatting tick marks and tick labels 
    ax1.tick_params(axis = 'both', labelsize = tick_fontsize)
#     ax1.ticklabel_format(axis = 'y', style = 'sci', scilimits = scilimits)
    # add rug plot at normal observations 
    sns.rugplot(pddf['vals'], ax = ax1, color = 'tab:blue')
    sns.rugplot(np.array([contam_data]), ax = ax1, color = 'red')

    save_dir_ml = (f'data/PenML-GD-ContamData={contam_data}-basisn=310-bw={bw}-kernel={kernel_type}-' + 
                   f'contamweight={contam_weight}-plotdomain={xlimit}-plotcnts={plot_pts_cnt}-seed={seed}')
 
    x = np.load(save_dir_ml + '/new_data.npy')
    y = np.load(save_dir_ml + f'/logpenparam={log_pen_param}-IF-newdata.npy')
    
    # plot log density when the basis functions are centered at grid points 
    ax1.plot(x.flatten(), y.flatten(), color = 'tab:red', linewidth = linewidth, 
            label = 'Maximum Likelihood')
    
    # draw a vertical line at the outlier 
    ax1.axvline(contam_data, 0, 1, ls = '--', color = 'tab:purple', alpha = 0.5)
    ax1.set_ylim(ylimit1)
    
#     # add grid
#     ax1.grid(color = 'k', ls = (0, (3, 10, 1, 10)), lw = 0.25)

    # add plot information 
    info = r'$\lambda$ = exp({log_pen})'.format(log_pen = log_pen_param)
    ax1.text(0.988, 0.988,
             info,
             fontsize = info_fontsize,
             # fontfamily = 'serif',
             multialignment = 'left',
             horizontalalignment = 'right',
             verticalalignment = 'top',
             transform = ax1.transAxes,
             bbox = {'facecolor': 'none',
                     'boxstyle': 'Round, pad=0.2'})

    # ---------------------------------------------------------------------------------------
    # set ax2 title 
    # ax2.set_title('Basis Functions Centered at Data Points', fontsize = fontsize_title)
    # set x-limit 
    ax2.set_xlim(xlimit)
    # set x label 
    ax2.set_xlabel(var_name, fontsize = label_fontsize)
    # set y label 
    ax2.set_ylabel('IF', fontsize = label_fontsize)
    # formatting tick marks and tick labels 
    ax2.tick_params(axis = 'both', labelsize = tick_fontsize)
    # add rug plot at normal observations 
    sns.rugplot(pddf['vals'], ax = ax2, color = 'tab:blue')
    sns.rugplot(np.array([contam_data]), ax = ax2, color = 'red')
    
    save_dir_sm = (f'data/PenSM-ContamData={contam_data}-bw={bw}-kernel={kernel_type}-' + 
                   f'contamweight={0.001}-plotdomain=(1.0, 310.0)-plotcnts=3000')
    
    x = np.load(save_dir_sm + '/new_data.npy')
    y = np.load(save_dir_sm + f'/logpenparam={log_pen_param}-IF-newdata.npy')
    
    ax2.plot(x.flatten(), y.flatten(), color = 'tab:blue', linewidth = linewidth, 
            label = 'Score Matching')
    
    # draw a vertical line at the outlier 
    ax2.axvline(contam_data, 0, 1, ls = '--', color = 'tab:purple', alpha = 0.5)
    ax2.set_ylim(ylimit2)
    
#     # add grid
#     ax2.grid(color = 'k', ls = (0, (3, 10, 1, 10)), lw = 0.25)
    
    # add plot information 
    info = r'$\lambda$ = exp({log_pen})'.format(log_pen = log_pen_param)
    ax2.text(0.988, 0.988,
             info,
             # fontfamily = 'serif',
             fontsize = info_fontsize,
             multialignment = 'left',
             horizontalalignment = 'right',
             verticalalignment = 'top',
             transform = ax2.transAxes,
             bbox = {'facecolor': 'none',
                     'boxstyle': 'Round, pad=0.2'})
    
    return ax1, ax2

ani = FuncAnimation(
    fig, 
    update_plots, 
    frames = log_pen_param_list, 
    interval = 500)

fig.suptitle(r'IF of ML and SM Log-density Estimates with $\sigma$={bw} and additional observation at {add_pt}'.format(
    bw=bw, add_pt = contam_data), 
             fontsize = fontsize_suptitle, y = 0.98)

# uncomment the following line to save the gif
ani.save(f'gif/PenML-PenSM-waiting-bw={bw}-contam_data={contam_data}.gif', writer='imagemagick')

plt.show()